In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OneHotEncoder
from sklearn.model_selection import train_test_split

In [2]:
df_train=pd.read_csv('/Users/abdul/Desktop/machine learning/data/home-data-for-ml-course/spaceship-titanic/train.csv')
df_test=pd.read_csv('/Users/abdul/Desktop/machine learning/data/home-data-for-ml-course/spaceship-titanic/test.csv')

In [7]:
df_train

,PassengerId,HomePlanet,CryoSleep,Cabin,Destination,Age,VIP,RoomService,FoodCourt,ShoppingMall,Spa,VRDeck,Name,Transported
0,0001_01,Europa,False,B/0/P,TRAPPIST-1e,39.0,False,0.0,0.0,0.0,0.0,0.0,Maham Ofracculy,False
1,0002_01,Earth,False,F/0/S,TRAPPIST-1e,24.0,False,109.0,9.0,25.0,549.0,44.0,Juanna Vines,True
2,0003_01,Europa,False,A/0/S,TRAPPIST-1e,58.0,True,43.0,3576.0,0.0,6715.0,49.0,Altark Susent,False
3,0003_02,Europa,False,A/0/S,TRAPPIST-1e,33.0,False,0.0,1283.0,371.0,3329.0,193.0,Solam Susent,False
4,0004_01,Earth,False,F/1/S,TRAPPIST-1e,16.0,False,303.0,70.0,151.0,565.0,2.0,Willy Santantines,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8688,9276_01,Europa,False,A/98/P,55 Cancri e,41.0,True,0.0,6819.0,0.0,1643.0,74.0,Gravior Noxnuther,False
8689,9278_01,Earth,True,G/1499/S,PSO J318.5-22,18.0,False,0.0,0.0,0.0,0.0,0.0,Kurta Mondalley,False
8690,9279_01,Earth,False,G/1500/S,TRAPPIST-1e,26.0,False,0.0,0.0,1872.0,1.0,0.0,Fayey Connon,True
8691,9280_01,Europa,False,E/608/S,55 Cancri e,32.0,False,0.0,1049.0,0.0,353.0,3235.0,Celeon Hontichre,False


In [8]:
def compte_na(x):
    # Calculer le nombre de valeurs manquantes par colonne
    missing_counts = x.isna().sum()
    # Calculer le pourcentage de valeurs manquantes par colonne
    percent_missing = (x.isna().sum() / x.shape[0]) * 100
    # Créer un DataFrame contenant les informations sur les colonnes avec des valeurs manquantes
    columns_with_missing_values = pd.DataFrame({
        'Nom': missing_counts.index,
        'Nbr manquants': missing_counts.values,
        'Pourcent manquants': percent_missing.values
    })
    # Filtrer les colonnes avec des valeurs manquantes
    columns_with_missing_values = columns_with_missing_values[columns_with_missing_values['Nbr manquants'] > 0]
    # Ordonner par ordre décroissant
    columns_with_missing_values=columns_with_missing_values.sort_values(by='Pourcent manquants', ascending=False)
    return columns_with_missing_values

In [10]:
compte_na(df_train)

,Nom,Nbr manquants,Pourcent manquants
2,CryoSleep,217,2.496261
9,ShoppingMall,208,2.392730
6,VIP,203,2.335212
1,HomePlanet,201,2.312205
12,Name,200,2.300702
3,Cabin,199,2.289198
11,VRDeck,188,2.162660
8,FoodCourt,183,2.105142
10,Spa,183,2.105142
4,Destination,182,2.093639
